In [1]:
import itertools
import numpy as np
import datetime
import pandas as pd   
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.metrics import accuracy_score
import gc
from sklearn.metrics import classification_report
from scipy.stats.stats import pearsonr

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import plotly.offline
import cufflinks as cf
import plotly.express as px
from plotly.offline import iplot
import plotly.graph_objects as go

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [3]:
path = '/home/brainiac/Vlad/Git Files/CryptoLp DataScience/Projects/09. Получение данных/01. Данные с BM/New Folder/trades'

d11 = pd.read_csv(f'{path}/2020/08/2020_08.csv')
d12 = pd.read_csv(f'{path}/2020/09/2020_09.csv')

sec_trades = pd.concat([d11,d12])
sec_trades.index = range(len(sec_trades))

In [4]:
sec_trades['diff'] = sec_trades['price'].diff(1)
sec_trades = sec_trades.dropna()

In [5]:
sec_trades.head(2)
sec_trades.tail(2)

,timestamp,side,size,price,tickDirection,diff
1,2020-08-01 00:00:02.904014,Sell,3000,11360.5,MinusTick,-0.5
2,2020-08-01 00:00:02.938797,Buy,15,11361.0,PlusTick,0.5


,timestamp,side,size,price,tickDirection,diff
19752821,2020-09-30 23:59:56.883595,Buy,849,10778.5,ZeroPlusTick,0.0
19752822,2020-09-30 23:59:59.975317,Buy,20180,10778.5,ZeroPlusTick,0.0


In [6]:
freq = '30s' 

sec_trades.index = sec_trades['timestamp'].values
sec_trades['timestamp'] = pd.to_datetime(sec_trades['timestamp'])    
df_Bt_bars = pd.DataFrame(columns=['datetime','price',])
    
df_Bt_bars['datetime'] = sec_trades.groupby(pd.Grouper(key='timestamp', freq=freq)).last().index
df_Bt_bars['dprice'] = sec_trades.groupby(pd.Grouper(key='timestamp', freq=freq)).sum()['diff'].values
df_Bt_bars['volum'] = sec_trades.groupby(pd.Grouper(key='timestamp', freq=freq)).sum()['size'].values

In [7]:
df_Bt_bars.tail(2)

,datetime,price,dprice,volum
175678,2020-09-30 23:59:00,NaN,-0.5,97471
175679,2020-09-30 23:59:30,NaN,8.5,1622150


In [8]:
sigma = df_Bt_bars['dprice'].std()

In [9]:
z = df_Bt_bars['dprice'].apply(lambda x: norm.cdf(x/sigma))

In [10]:
df = pd.DataFrame({'z': z, 'volume': df_Bt_bars['volum']}).dropna()

In [11]:
df.head()

,z,volume
0,0.607093,1310863
1,0.999929,3345387
2,0.996711,4806521
3,0.861481,698060
4,0.792535,1969620


In [12]:
df['z'].nunique()

253

In [13]:
df.head()

,z,volume
0,0.607093,1310863
1,0.999929,3345387
2,0.996711,4806521
3,0.861481,698060
4,0.792535,1969620


In [14]:
df.index = df_Bt_bars['datetime']

In [15]:
df

,z,volume
datetime,,
2020-08-01 00:00:00,0.607093,1310863
2020-08-01 00:00:30,0.999929,3345387
2020-08-01 00:01:00,0.996711,4806521
2020-08-01 00:01:30,0.861481,698060
2020-08-01 00:02:00,0.792535,1969620
...,...,...
2020-09-30 23:57:30,0.500000,2259
2020-09-30 23:58:00,0.500000,13812
2020-09-30 23:58:30,0.536088,26680


### До этого момента все работает

In [16]:
volumeBuckets = pd.DataFrame(columns=['Buy','Sell','Time'])
count = 0
BV = 0
SV = 0

In [17]:
%%time
bucketSize = 100_000
for index,row in df.iterrows():
    newVolume = row['volume']
    z = row['z']
    if bucketSize < count + newVolume:
        BV = BV + (bucketSize-count)*z
        SV = SV + (bucketSize-count)*(1-z)
        volumeBuckets = volumeBuckets.append({'Buy':BV, 'Sell':SV, 'Time':index},ignore_index=True)
        count = newVolume-(bucketSize-count)
        if int(count/bucketSize) > 0:
                for i in range(0,int(count/bucketSize)):
                    BV = (bucketSize)*z
                    SV = (bucketSize)*(1-z)
                    volumeBuckets = volumeBuckets.append({'Buy':BV, 'Sell':SV, 'Time':index},ignore_index=True)

        count = count%bucketSize
        BV = (count)*z
        SV = (count)*(1-z)
    else:
        BV = BV + (newVolume)*z
        SV = SV + (newVolume)*(1-z)
        count = count + newVolume

volumeBuckets = volumeBuckets.set_index('Time')

CPU times: user 46min 11s, sys: 5.94 s, total: 46min 17s
Wall time: 46min 20s


In [18]:
volumeBuckets

,Buy,Sell
Time,,
2020-08-01 00:00:00,60709.255501,39290.744499
2020-08-01 00:00:00,60709.255501,39290.744499
2020-08-01 00:00:00,60709.255501,39290.744499
2020-08-01 00:00:00,60709.255501,39290.744499
2020-08-01 00:00:00,60709.255501,39290.744499
...,...,...
2020-09-30 23:59:30,93820.909400,6179.090600
2020-09-30 23:59:30,93820.909400,6179.090600
2020-09-30 23:59:30,93820.909400,6179.090600


In [19]:
window = 10
#volumeBuckets['VPIN'] = abs(volumeBuckets['Buy']-volumeBuckets['Sell']).rolling(window).mean()/bucketSize

volumeBuckets['VPIN'] = volumeBuckets['Buy']-volumeBuckets['Sell'].rolling(window).mean()/bucketSize

volumeBuckets['CDF'] = volumeBuckets['VPIN'].rank(pct=True)

In [20]:
volumeBuckets.tail()

,Buy,Sell,VPIN,CDF
Time,,,,
2020-09-30 23:59:30,93820.9094,6179.0906,93820.847609,0.826207
2020-09-30 23:59:30,93820.9094,6179.0906,93820.847609,0.826207
2020-09-30 23:59:30,93820.9094,6179.0906,93820.847609,0.826207
2020-09-30 23:59:30,93820.9094,6179.0906,93820.847609,0.826207
2020-09-30 23:59:30,93820.9094,6179.0906,93820.847609,0.826207


In [21]:
volumeBuckets['datetime'] = volumeBuckets.index
volumeBuckets = volumeBuckets.dropna()
volumeBuckets.index = range(len(volumeBuckets))

In [22]:
volumeBuckets['VPIN'].quantile(0.8)

91288.19524287526

In [23]:
list_vpin = volumeBuckets['VPIN'].values
list_signal = list()
for i in range(len(volumeBuckets)):
    if list_vpin[i] >= 0.85:
        list_signal.append('Signal')
    else:
        list_signal.append('Unchanged')

In [26]:
volumeBuckets['Signal Toxity'] = list_signal

In [27]:
volumeBuckets.head()

,Buy,Sell,VPIN,CDF,datetime,Sinal Toxity,Signal Toxity
0,60709.255501,39290.744499,0.214185,0.154041,2020-08-01 00:00:00,Unchanged,Unchanged
1,60709.255501,39290.744499,0.214185,0.154041,2020-08-01 00:00:00,Unchanged,Unchanged
2,60709.255501,39290.744499,0.214185,0.154041,2020-08-01 00:00:00,Unchanged,Unchanged
3,60709.255501,39290.744499,0.214185,0.154041,2020-08-01 00:00:00,Unchanged,Unchanged
4,95725.513350,4274.486650,0.284218,0.215069,2020-08-01 00:00:30,Unchanged,Unchanged


In [28]:
volumeBuckets[['datetime', 'Signal Toxity']].to_csv('/home/brainiac/Vlad/Git Files/CryptoLp DataScience/Projects/Untitled Folder/Токсичность.csv', index=False)

In [88]:
volumeBuckets[volumeBuckets['VPIN']>volumeBuckets['VPIN'].quantile(0.8)].iloc[142:332]

,Buy,Sell,VPIN,CDF
Time,,,,
2020-11-01 00:08:30,1771.279288,98228.720712,0.964574,0.817792
2020-11-01 00:32:30,50.692279,99949.307721,0.952444,0.803475
2020-11-01 00:32:30,50.692279,99949.307721,0.998986,0.900903
2020-11-01 00:32:30,50.692279,99949.307721,0.998986,0.901016
2020-11-01 00:32:30,50.692279,99949.307721,0.998986,0.901016
...,...,...,...,...
2020-11-01 00:38:00,99780.326073,219.673927,0.995607,0.877804
2020-11-01 00:38:00,99780.326073,219.673927,0.995607,0.877804
2020-11-01 00:38:00,99780.326073,219.673927,0.995607,0.877804


In [23]:
OrderBook_BM = pd.read_csv('/home/brainiac/Vlad/Git Files/CryptoLp DataScience/Projects/09. Получение данных/01. Данные с BM/New Folder/quote.csv')

In [37]:
OrderBook_BM.head(2)

,timestamp,bidSize,bidPrice,askPrice,askSize
0,2020-11-01 00:00:02.304829,4601.0,13802.0,13802.5,1424554.0
1,2020-11-01 00:00:02.313976,4601.0,13802.0,13802.5,1423554.0


In [31]:
fig = go.Figure()

part = OrderBook_BM[(OrderBook_BM['timestamp']>="2020-11-01 00:00:02.304829") &
             (OrderBook_BM['timestamp']<="2020-11-01 00:01:02.304829")]

fig.add_trace(go.Scatter(x=part['timestamp'], y=part['bidPrice'].values,
                         mode='markers', name='Bid'))
fig.add_trace(go.Scatter(x=part['timestamp'], y=part['askPrice'].values,
                         mode='markers', name='Ask'))
plotly.offline.plot(fig, filename='1.html')

'1.html'